<a href="https://colab.research.google.com/github/suvarnak10/c4h_testing/blob/main/LLM_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain-groq pymupdf
!pip install pymupdf requests
!pip install faiss-cpu sentence-transformers pymupdf langchain_groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 867.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import os
import pickle
import pymupdf  # PDF processing
import faiss  # Vector search
import numpy as np
from sentence_transformers import SentenceTransformer
from langchain_groq import ChatGroq

In [5]:
# Set your Groq API Key
API_KEY="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"#groq api

# Load embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# FAISS index for vector storage
dimension = 384  # Embedding size for MiniLM
index = faiss.IndexFlatL2(dimension)
text_chunks = []  # Store original text chunks


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pymupdf.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text("text") + "\n"
    except Exception as e:
        print(f"Error reading PDF: {e}")
    return text

In [7]:
# Function to split text into chunks
def split_text_into_chunks(text, max_chars=3500):
    chunks = []
    while len(text) > max_chars:
        split_index = text[:max_chars].rfind(".")
        if split_index == -1:
            split_index = max_chars
        chunks.append(text[:split_index + 1])
        text = text[split_index + 1:]
    chunks.append(text)
    return chunks

In [8]:
# Function to embed and index text in FAISS
def index_pdf_text(pdf_text):
    global text_chunks
    chunks = split_text_into_chunks(pdf_text, max_chars=3500)
    text_chunks = chunks  # Store for retrieval

    embeddings = embedding_model.encode(chunks)  # Convert text to embeddings
    index.add(np.array(embeddings, dtype=np.float32))  # Add to FAISS

    print(f"Indexed {len(chunks)} chunks in FAISS.")

In [9]:
# Function to search similar chunks in FAISS
def search_relevant_chunks(query, top_k=3):
    query_embedding = embedding_model.encode([query]).astype(np.float32)
    _, indices = index.search(query_embedding, top_k)

    retrieved_chunks = [text_chunks[i] for i in indices[0] if i < len(text_chunks)]
    return " ".join(retrieved_chunks) if retrieved_chunks else "No relevant data found."


In [10]:
def analyze_query_with_groq(relevant_text, query):
    llm = ChatGroq(api_key=API_KEY, model="mixtral-8x7b-32768")

    prompt = f"""
    You are an AI financial analyst. Given the following **earnings call transcript**, extract key investor insights.

    **Instructions:**
    - Summarize the most relevant financial details for an investor.
    - Provide structured insights under these categories:
      1. **Future Growth Prospects**
      2. **Key Business Changes**
      3. **Key Triggers**
      4. **Financial Outlook**

    **Relevant Transcript Excerpt:**
    {relevant_text}

    **Investor Query:** {query}

    **Format Output as:**
    **Future Growth Prospects:** [Details]
    **Key Business Changes:** [Details]
    **Key Triggers:** [Details]
    **Financial Outlook:** [Details]
    """

    try:
        response = llm.invoke(prompt)
        return response.content if response else "No response received."
    except Exception as e:
        return f"Error in Groq API call: {e}"

In [11]:
def process_pdf_and_query(investor_query):
    relevant_text = search_relevant_chunks(investor_query)  # Retrieve relevant chunks

    response = analyze_query_with_groq(relevant_text, investor_query)  # Get AI insights
    return response

In [12]:
# Example usage
pdf_file = "/content/SJS Transcript Call.pdf"
query = "What are the key growth prospects for the company?"
pdf_text = extract_text_from_pdf(pdf_file)
if not pdf_text:
    print("No text extracted from the PDF.")
index_pdf_text(pdf_text)

# Process the PDF and get an investor-focused response
investor_insights = process_pdf_and_query(query)

# Print extracted insights
print("\n### Key Investor Insights ###\n")
print(investor_insights)

Indexed 16 chunks in FAISS.

### Key Investor Insights ###

**Future Growth Prospects:**
The company expects a consolidated growth of 50% for FY24, including the contribution from Walter Pack India (WPI) acquisition. The organic growth of SJS and Exotech is expected to be 20% to 25%. WPI is projected to grow at the same rate as the overall business, and it will be EPS accretive for the company. New product development contributes around 10% to the consolidated revenue, and the company aims to increase its value in two-wheeler, passenger vehicle, and consumer segments with new technology premium products.

**Key Business Changes:**
The acquisition of WPI is a significant change in the company's portfolio, providing access to new technologies and markets. Exotech's chrome plating capacity expansion CAPEX plan has been deferred by a year to align with evolving business requirements of both Exotech and WPI. In the interim, Exotech's capacity will be increased through debottlenecking, under

In [15]:
investor_insights = process_pdf_and_query("who is Mr. K A Joseph and Mr. Sanjay Thapar?")#add different queries to test

In [16]:
print(investor_insights)

**Future Growth Prospects:**
- SJS Enterprises is executing its organic growth strategy by addressing the aesthetic requirements of multiple end-industry segments, increasing global presence, introducing new technology products, and building mega customer accounts. This strategy has helped the company outperform underlying industry growth rates.
- The company has identified cross-selling opportunities, synergies, and growing economies of scale between its recent acquisition, Walter Pack India, and its existing businesses, Exotech and Walter Pack.
- SJS is increasing its addressable market by introducing new premium products and technologies that are complex to manufacture, such as chrome plated parts, IML, IMD, and IMF parts, and new age products like optical plastics and cover glass.

**Key Business Changes:**
- SJS has deferred its chrome plating capacity expansion CAPEX plan by a year to calendar year '24 due to synergies between Exotech and Walter Pack. In the interim, Exotech's ca